# HRNet_Classification

In [6]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
import keras
# import keras.backend as K
from keras.layers import Conv2D, BatchNormalization, Activation, Dense, Input, Add, GlobalAveragePooling2D
from keras.models import Model
from HRNet import HRNet, Strided_Conv_Block


print(tf.__version__)       # 2.11.0
print(keras.__version__)    # 2.11.0


def HRNet_Classification(input_shape, n_classes, n_filters):

  inputs = Input(input_shape)

  stage1, stage2, stage3, stage4 = HRNet(inputs, n_filters=n_filters)

  stage1 = Conv2D(filters=4*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage1)
  stage1 = BatchNormalization()(stage1)
  stage1 = Activation('relu')(stage1)

  stage2 = Conv2D(filters=8*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage2)
  stage2 = BatchNormalization()(stage2)
  stage2 = Activation('relu')(stage2)

  stage3 = Conv2D(filters=16*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage3)
  stage3 = BatchNormalization()(stage3)
  stage3 = Activation('relu')(stage3)

  stage4 = Conv2D(filters=32*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(stage4)
  stage4 = BatchNormalization()(stage4)
  stage4 = Activation('relu')(stage4)

  y1 = Strided_Conv_Block(stage1, 8*n_filters, n_stride=2)
  y = Add()([stage2, y1])

  y2 = Strided_Conv_Block(y1, 16*n_filters, n_stride=2)
  y = Add()([stage3, y2])

  y3 = Strided_Conv_Block(y2, 32*n_filters, n_stride=2)
  y = Add()([stage4, y3])

  y = Conv2D(filters=64*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(y)
  y = BatchNormalization()(y)
  y = Activation('relu')(y)

  y = Conv2D(filters=64*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(y)
  y = BatchNormalization()(y)
  y = Activation('relu')(y)

  y = GlobalAveragePooling2D()(y)
  y = Dense(n_classes, use_bias=False)(y)

  if n_classes == 1:
    y = Activation('sigmoid')(y)
  else:
    y = Activation('softmax')(y)
  # print(y.shape)

  model = Model(inputs=inputs, outputs=y)

  return model

def main():

  model = HRNet_Classification(input_shape=(224,224,3), n_classes=4, n_filters=16)
  # model.summary()  
  model.save('sdaads.h5')

if __name__== '__main__':

  main()

2.11.0
2.11.0
(None, 224, 224, 16) (None, 112, 112, 32) (None, 56, 56, 64) (None, 28, 28, 128)


# HRNet_Segmentation

In [15]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
import keras
# import keras.backend as K
from keras.layers import Conv2D, BatchNormalization, Activation, Input, UpSampling2D, Concatenate
from keras.models import Model
from HRNet import HRNet, Strided_Conv_Block


print(tf.__version__)       # 2.11.0
print(keras.__version__)    # 2.11.0


def HRNet_Segmentation(input_shape, n_classes, n_filters):

  inputs = Input(input_shape)

  stage1, stage2, stage3, stage4 = HRNet(inputs, n_filters=n_filters)

  stage2 = UpSampling2D(size=2, interpolation='bilinear')(stage2)
  stage3 = UpSampling2D(size=4, interpolation='bilinear')(stage3)
  stage4 = UpSampling2D(size=8, interpolation='bilinear')(stage4)
  
  y = Concatenate()([stage1, stage2, stage3, stage4])

  y = Conv2D(filters=2*n_filters, kernel_size=1, strides=1, padding='same', use_bias=False)(y)
  y = BatchNormalization()(y)
  y = Activation('relu')(y)

  y = Conv2D(filters=n_classes, kernel_size=1, strides=1, padding='same', use_bias=False)(y)

  if n_classes == 1:
    y = Activation('sigmoid')(y)
  else:
    y = Activation('softmax')(y)
  # print(y.shape)

  model = Model(inputs=inputs, outputs=y)

  return model

def main():

  model = HRNet_Segmentation(input_shape=(512,512,3), n_classes=10, n_filters=16)
  # model.summary()  
  model.save('sdaads.h5')

if __name__== '__main__':

  main()

2.11.0
2.11.0


(None, 512, 512, 16) (None, 256, 256, 32) (None, 128, 128, 64) (None, 64, 64, 128)
